In [0]:
# Databricks notebook source
try:
    dbutils.fs.unmount("/mnt/input")
except:
    print("Input Mount does not exist")

try:
    dbutils.fs.unmount("/mnt/output1")
except:
    print("Output Mount does not exist")

input_source = dbutils.widgets.get("input")
output1 = dbutils.widgets.get("testdata")

# conf_key = "fs.azure.account.key.<your storage account name>.blob.core.windows.net"
# key_value = "<Storage account access key>"
conf_key = "fs.azure.account.key.dp100udemy.blob.core.windows.net"
key_value = "32cMFMxXzFskFrYFo84e1etBWDDZvUecgxgsXR7w+14M73Ih5lZqVN2VCoTAWp3oDqtMWI6QQCog+AStU5YYbQ=="

try: 
    dbutils.fs.mount(source = input_source,
                   mount_point = "/mnt/input",
                   extra_configs = {conf_key:key_value})
except:
    print('Input datastore already mounted...')

try:
    dbutils.fs.mount(source = output1,
                   mount_point = "/mnt/output1",
                   extra_configs = {conf_key:key_value})
except:
    print("Output data folder already mounted...")

Input Mount does not exist
Output Mount does not exist

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-883396618295869> in <module> 
 10 print ( "Output Mount does not exist" ) 
 11 
 ---> 12 input_source = dbutils . widgets . get ( "input" ) 
 13 output1 = dbutils . widgets . get ( "testdata" ) 
 14 

 /databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get (self, name) 
 40 : return : Current value of the widget or default value
 41 """
 ---> 42 return self . _notebookArguments . getArgument ( name , self . _entry_point . getCurrentBindings ( ) ) 
 43 
 44 def getArgument ( self , name , defaultValue = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o327.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named input is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
import pandas as pd

# storageアカウントのコンテナのdataフォルダの中にファイルを格納している 
df = pd.read_csv("/dbfs/mnt/input/data/adultincome trunc.csv")

# Create Dummy variables
data_prep = pd.get_dummies(df, drop_first=True)

# Create X and Y Variables
X = data_prep.iloc[:, :-1]
Y = data_prep.iloc[:, -1]

# Split the X and Y dataset into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1234, stratify=Y)

# Import and train Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1234)

trained_model = rfc.fit(X_train, Y_train)

# Test the RFC model
Y_predict = rfc.predict(X_test)

In [0]:
# フォルダを作成
dbutils.fs.mkdirs("/mnt/output1/000")

import joblib
obj_file = "/dbfs/mnt/output1/rfcModel.pkl"
joblib.dump(value=trained_model, filename=obj_file)

# Y_test, Y_predict
X_test.to_csv("/dbfs/mnt/output1/x_test.csv", index=False)

Y_test = pd.DataFrame(Y_test)
Y_test.to_csv("/dbfs/mnt/output1/y_test.csv", index=False)

Y_predict = pd.DataFrame(Y_predict)
Y_predict.to_csv("/dbfs/mnt/output1/y_predict.csv", index=False)